In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def prd_calc(prd,Data):
    
    # Takes prd (integer) is the period were we need to find the data   #
    # Takes Data (Dataframe) is the data                                #
    # Return the average of every sector in this period based on spread #                                                         #
    dist=(Data.rind).value_counts()
    dist.index
    
    DataDay_one=[]
    for i in range(0,len(dist.index)):
        Day=Data.loc[Data['prd']==prd]
        Sector=Day.loc[Day['rind']==dist.index[i]]
        tret=Sector['tret']
        tret.to_numpy()
        avg_tret=np.average(tret)
        DataDay_one.append(avg_tret)
        
    return DataDay_one

In [7]:
def Values_from_correlation(ind1,ind2,corr):
    return corr[ind1][ind2]

In [8]:
def calculate_matricies(tick_ind,corr):
    corr_by_int=[]
    for int_i,sector_i in zip(tick_ind['int_tick'],tick_ind['rind']):
        lines=[]
        for int_j , sector_j in zip(tick_ind['int_tick'],tick_ind['rind']):
            lines.append(Values_from_correlation(sector_i,sector_j,corr))
        corr_by_int.append(lines)
    corr_by_int=pd.DataFrame(corr_by_int)
    corr_by_int.index=list(tick_ind['int_tick'])
    corr_by_int.columns=list(tick_ind['int_tick'])
    return corr_by_int

In [9]:
# import the data
Data=pd.read_csv('preproccesed.csv')
Data=Data.drop('Unnamed: 0',axis=1)

# Keep the data valuable for correlation
prd=Data.prd
int_tick=Data.int_tick
rind=Data.rind
tret=Data.tret
formal=pd.concat([int_tick,prd,rind,tret],axis=1)

In [10]:
# list of periods
periods=list((formal.loc[formal['int_tick']==int_tick[0]]).prd)

# for every period
SectPrd=[]
for i in range(0,len(periods)):
    SectPrd.append(prd_calc(periods[i],formal))

Sector_series=np.array(SectPrd) 
Sector_series=pd.DataFrame(Sector_series)

# Names of industries
Names_of_inds=list(((formal.rind).value_counts()).index)
Sector_series.columns=Names_of_inds


In [11]:
Sector_series

,non_cycl_HG,energy_HG,low_cyc_HY,basics_HG,util_HG,energy_HY,cap_g_HG,basics_HY,leis_trav_HY,auto_tech_HG,telco_HG,telco_HY,rtl_leis_HG,non_cycl_SG,auto_tech_HY,cycl_SG
0,0.000196,-0.000561,0.002202,0.000038,-0.001138,0.000839,0.000085,-0.000043,0.003874,0.000038,0.000746,0.002417,0.001496,-0.001444,0.001178,-0.000088
1,0.000298,-0.000101,0.001672,0.000624,-0.000855,0.001928,0.000059,0.000705,0.003992,-0.000062,0.000204,0.003154,0.001190,-0.000596,0.003079,-0.000059
2,0.000245,0.000017,0.001245,0.000454,-0.000424,0.001322,0.000105,0.000559,0.002560,0.000042,0.000217,0.002077,0.000770,-0.000311,0.002011,-0.000346
3,0.000116,0.000141,0.001531,0.000557,-0.000242,0.002649,0.000052,0.001901,0.003056,0.000209,0.000507,0.001659,0.000421,-0.000192,0.002786,-0.000162
4,0.000139,0.000163,0.001670,0.000472,-0.000088,0.002105,0.000081,0.002865,0.003321,0.000488,0.000485,0.001004,0.000585,-0.000221,0.004339,-0.000174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.000632,0.000786,0.002505,0.001012,0.000507,0.002039,0.000596,0.002184,0.002570,0.000665,0.000444,0.002076,0.000795,0.000251,0.001477,0.000278
77,0.000581,0.000757,0.002534,0.001008,0.000421,0.002099,0.000605,0.002378,0.002632,0.000639,0.000362,0.002135,0.000872,0.000182,0.001457,0.000226
78,0.000563,0.000669,0.002557,0.001225,0.000371,0.002148,0.000568,0.002537,0.002682,0.000591,0.000312,0.002184,0.001577,0.000149,0.001440,0.000174
79,0.000515,0.000740,0.002576,0.000966,0.000505,0.002188,0.000499,0.002666,0.002724,0.000537,0.000288,0.002223,0.001424,0.000110,0.001427,0.000304


In [12]:
# calculate correlation by window
corr_prd=[]
steps=[]
end=0
for start in range(0,len(Sector_series)):
    if end<len(Sector_series):
        # for every window
        end=start+10
        step=Sector_series.loc[start:end]
        print(end)
        steps.append(step)
        step_corr=step.corr()
        # Make zeros those below zero
        step_corr[step_corr<0]=0
        # Normalize correlation map
        step_corr = np.array(step_corr)   
        step_corr=(step_corr-step_corr.min())/(step_corr.max()-step_corr.min()) 
        # Take the 1-corr
        step_corr=pd.DataFrame(step_corr)
        step_corr=1-step_corr

        step_corr.columns=Names_of_inds
        step_corr.index=Names_of_inds

        corr_prd.append(step_corr)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


In [13]:
sample_day=formal.loc[formal['prd']==periods[0]]
int_tick=sample_day.int_tick
rind=sample_day.rind
int_tick_ind=pd.concat([int_tick,rind],axis=1)
int_tick_ind=int_tick_ind.reset_index(drop=True)

In [14]:
time_series_corr=[]
iter_=1
for corr in corr_prd:
    print(iter_)
    time_series_corr.append(calculate_matricies(int_tick_ind,corr))
    iter_=iter_+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [15]:
use_periods=periods[10:end+1]

In [16]:
for prd,corr in zip(use_periods,time_series_corr):
    corr.to_csv(str(prd)+' rint-tret.csv')